# Training Module

## Import Libraries

In [1]:
import pandas as pd
from nba_api.stats.endpoints import playercareerstats
from nba_api.stats.endpoints import commonallplayers

## Load Data

In [126]:
# Grab the active players from the 2024-25 season
all_players_2024 = commonallplayers.CommonAllPlayers(season='2024-25').get_data_frames()[0]
active_players = all_players_2024[all_players_2024['ROSTERSTATUS'] == 1]
active_player_ids = list(active_players['PERSON_ID'])


In [132]:
active_players.head()

,PERSON_ID,DISPLAY_LAST_COMMA_FIRST,DISPLAY_FIRST_LAST,ROSTERSTATUS,FROM_YEAR,TO_YEAR,PLAYERCODE,PLAYER_SLUG,TEAM_ID,TEAM_CITY,TEAM_NAME,TEAM_ABBREVIATION,TEAM_CODE,TEAM_SLUG,GAMES_PLAYED_FLAG,OTHERLEAGUE_EXPERIENCE_CH
10,1630173,"Achiuwa, Precious",Precious Achiuwa,1,2020,2024,precious_achiuwa,precious_achiuwa,1610612752,New York,Knicks,NYK,knicks,knicks,Y,00
22,203500,"Adams, Steven",Steven Adams,1,2013,2024,steven_adams,steven_adams,1610612745,Houston,Rockets,HOU,rockets,rockets,Y,00
24,1628389,"Adebayo, Bam",Bam Adebayo,1,2017,2024,bam_adebayo,bam_adebayo,1610612748,Miami,Heat,MIA,heat,heat,Y,00
29,1630534,"Agbaji, Ochai",Ochai Agbaji,1,2022,2024,ochai_agbaji,ochai_agbaji,1610612761,Toronto,Raptors,TOR,raptors,raptors,Y,00
41,1630583,"Aldama, Santi",Santi Aldama,1,2021,2024,santi_aldama,santi_aldama,1610612763,Memphis,Grizzlies,MEM,grizzlies,grizzlies,Y,01


In [127]:
def create_active_player_stats(player_ids):
    active_player_stats = {'PLAYER_ID': [], 'FG_PCT': [], 'FT_PCT': [], '3PM': [], 'PTS': [], 'REB': [], 'AST': [], 'STL': [], 'BLK': [], 'TOV': []}
    for player_id in player_ids:
        career = playercareerstats.PlayerCareerStats(player_id=player_id).get_data_frames()[0]
        career = career[career['SEASON_ID'].isin(['2022-23', '2023-24'])]
        # Aggregate the stats for the player
        active_player_stats['PLAYER_ID'].append(player_id)
        active_player_stats['FG_PCT'].append(career['FG_PCT'].mean())
        active_player_stats['FT_PCT'].append(career['FT_PCT'].mean())
        active_player_stats['3PM'].append(career['FG3M'].mean())
        active_player_stats['PTS'].append(career['PTS'].mean())
        active_player_stats['REB'].append(career['REB'].mean())
        active_player_stats['AST'].append(career['AST'].mean())
        active_player_stats['STL'].append(career['STL'].mean())
        active_player_stats['BLK'].append(career['BLK'].mean())
        active_player_stats['TOV'].append(career['TOV'].mean())
    return pd.DataFrame(active_player_stats)

In [128]:
active_player_stats = create_active_player_stats(active_player_ids)

In [133]:
active_player_stats['PLAYER_NAME'] = active_player_stats['PLAYER_ID'].apply(lambda x: active_players[active_players['PERSON_ID'] == x]['DISPLAY_FIRST_LAST'].values[0])

In [136]:
# Move the last column to the front
cols = list(active_player_stats)
cols.insert(0, cols.pop(cols.index('PLAYER_NAME')))
active_player_stats = active_player_stats.loc[:, cols]

In [138]:
active_player_stats.to_csv('data/active_player_stats.csv', index=False)

## 9-Cat Ranking

In [30]:
active_player_stats = pd.read_csv('../data/active_player_stats.csv')
active_player_stats.head()

,PLAYER_NAME,PLAYER_ID,FG_PCT,FT_PCT,3PM,PTS,REB,AST,STL,BLK,TOV
0,Precious Achiuwa,1630173,0.49250,0.6330,20.25,409.50,325.50,61.00,30.75,41.50,56.25
1,Steven Adams,203500,0.59700,0.3640,0.00,361.00,485.00,97.00,36.00,46.00,79.00
2,Bam Adebayo,1628389,0.53050,0.7805,8.00,1448.00,712.50,259.00,84.50,63.50,174.50
3,Ochai Agbaji,1630534,0.41375,0.7085,51.25,344.25,138.25,58.25,27.50,25.75,42.25
4,Santi Aldama,1630583,0.45250,0.6855,100.00,675.00,361.50,117.50,44.00,51.00,64.50


In [31]:
def gen_z_score(row, pop_stats):
    """Compute the z-score for a given player"""
    z_score = 0
    for col in pop_stats.keys():
        z_score += (row[col] - pop_stats[col]['mean']) / pop_stats[col]['sd']
    return z_score

    

In [32]:
def gen_agg_z_score(player_stats):
    # Cat 1: Points
    player_ranked_by_pts = player_stats.sort_values(by='PTS', ascending=False).iloc[:200]
    pop_mean_pts = player_ranked_by_pts['PTS'].mean()
    pop_sd_pts = player_ranked_by_pts['PTS'].std()
    # Cat 2: FG %
    player_ranked_by_fg_pct = player_stats.sort_values(by='FG_PCT', ascending=False).iloc[:200]
    pop_mean_fg_pct = player_ranked_by_fg_pct['FG_PCT'].mean()
    pop_sd_fg_pct = player_ranked_by_fg_pct['FG_PCT'].std()
    # Cat 3: FT %
    player_ranked_by_ft_pct = player_stats.sort_values(by='FT_PCT', ascending=False).iloc[:200]
    pop_mean_ft_pct = player_ranked_by_ft_pct['FT_PCT'].mean()
    pop_sd_ft_pct = player_ranked_by_ft_pct['FT_PCT'].std()
    # Cat 4: Threes
    player_ranked_by_threes = player_stats.sort_values(by='3PM', ascending=False).iloc[:200]
    pop_mean_threes = player_ranked_by_threes['3PM'].mean()
    pop_sd_threes = player_ranked_by_threes['3PM'].std()
    # Cat 5: Assists
    player_ranked_by_asts = player_stats.sort_values(by='AST', ascending=False).iloc[:200]
    pop_mean_asts = player_ranked_by_asts['AST'].mean()
    pop_sd_asts = player_ranked_by_asts['AST'].std()
    # Cat 6: Rebounds
    player_ranked_by_rebs = player_stats.sort_values(by='REB', ascending=False).iloc[:200]
    pop_mean_rebs = player_ranked_by_rebs['REB'].mean()
    pop_sd_rebs = player_ranked_by_rebs['REB'].std()
    # Cat 7: Steals
    player_ranked_by_stls = player_stats.sort_values(by='STL', ascending=False).iloc[:200]
    pop_mean_stls = player_ranked_by_stls['STL'].mean()
    pop_sd_stls = player_ranked_by_stls['STL'].std()
    # Cat 8: Blocks
    player_ranked_by_blks = player_stats.sort_values(by='BLK', ascending=False).iloc[:200]
    pop_mean_blks = player_ranked_by_blks['BLK'].mean()
    pop_sd_blks = player_ranked_by_blks['BLK'].std()
    # Cat 9: TOs
    player_ranked_by_tovs = player_stats.sort_values(by='TOV', ascending=False).iloc[:200]
    pop_mean_tovs = player_ranked_by_tovs['TOV'].mean()
    pop_sd_tovs = player_ranked_by_tovs['TOV'].std()
    
    # Add the population stats to a dictionary
    pop_stats = {'PTS': {'mean': pop_mean_pts, 'sd': pop_sd_pts},
                 'FG_PCT': {'mean': pop_mean_fg_pct, 'sd': pop_sd_fg_pct},
                 'FT_PCT': {'mean': pop_mean_ft_pct, 'sd': pop_sd_ft_pct},
                 '3PM': {'mean': pop_mean_threes, 'sd': pop_sd_threes},
                 'AST': {'mean': pop_mean_asts, 'sd': pop_sd_asts},
                 'REB': {'mean': pop_mean_rebs, 'sd': pop_sd_rebs},
                 'STL': {'mean': pop_mean_stls, 'sd': pop_sd_stls},
                 'BLK': {'mean': pop_mean_blks, 'sd': pop_sd_blks},
                 'TOV': {'mean': pop_mean_tovs, 'sd': pop_sd_tovs}}
    # Compute the aggregate z-score
    player_stats['Z_SCORE'] = player_stats.apply(lambda row: gen_z_score(row, pop_stats), axis=1)
    return player_stats

In [33]:
# Generate the aggregate z-score for each player
active_player_stats_w_z_score = gen_agg_z_score(active_player_stats)

In [34]:
active_player_stats.head()

,PLAYER_NAME,PLAYER_ID,FG_PCT,FT_PCT,3PM,PTS,REB,AST,STL,BLK,TOV,Z_SCORE
0,Precious Achiuwa,1630173,0.49250,0.6330,20.25,409.50,325.50,61.00,30.75,41.50,56.25,-13.469413
1,Steven Adams,203500,0.59700,0.3640,0.00,361.00,485.00,97.00,36.00,46.00,79.00,-16.022628
2,Bam Adebayo,1628389,0.53050,0.7805,8.00,1448.00,712.50,259.00,84.50,63.50,174.50,2.699556
3,Ochai Agbaji,1630534,0.41375,0.7085,51.25,344.25,138.25,58.25,27.50,25.75,42.25,-14.701338
4,Santi Aldama,1630583,0.45250,0.6855,100.00,675.00,361.50,117.50,44.00,51.00,64.50,-8.770833


In [38]:
# Sort the players by z-score in descending order
active_player_stats_w_z_score = active_player_stats_w_z_score.sort_values(by='Z_SCORE', ascending=False)
active_player_stats_w_z_score.to_csv('../data/active_player_stats_w_z_score.csv', index=False)

In [39]:
active_player_stats_w_z_score.iloc[:200]

,PLAYER_NAME,PLAYER_ID,FG_PCT,FT_PCT,3PM,PTS,REB,AST,STL,BLK,TOV,Z_SCORE
232,Nikola Jokić,203999,0.60750,0.81950,70.00,1887.50,896.50,693.00,97.50,57.50,242.00,13.684092
489,Victor Wembanyama,1641705,0.46500,0.79600,128.00,1522.00,755.00,274.00,88.00,254.00,260.00,12.797552
113,Luka Dončić,1629029,0.49150,0.76400,234.50,2254.00,608.00,607.50,94.50,35.50,259.00,12.128554
153,Shai Gilgeous-Alexander,1628983,0.52250,0.88950,76.50,2194.50,372.00,418.00,131.00,66.00,177.00,10.041760
128,Anthony Edwards,1630162,0.46000,0.79600,201.50,1997.50,444.00,377.50,113.00,50.00,250.00,9.368886
...,...,...,...,...,...,...,...,...,...,...,...,...
27,RJ Barrett,1629628,0.47625,0.72875,75.25,942.75,249.50,146.75,23.25,14.75,104.00,-9.142815
195,Talen Horton-Tucker,1629659,0.40750,0.77850,59.00,607.00,165.50,211.50,43.00,24.00,95.50,-9.166685
41,Bol Bol,1629626,0.58100,0.77400,26.00,428.50,271.00,43.00,20.00,55.00,65.50,-9.375760
358,Mason Plumlee,203486,0.66125,0.68000,0.00,489.50,408.50,136.25,26.50,27.75,67.50,-9.377169
